In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
root_path='/content/gdrive/MyDrive/Datasets/insulator_data/Defective_Insulators/images/'

In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
from pathlib import Path
import glob
import os
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import tensorflow_hub as hub

In [ ]:
img = cv2.imread('/content/gdrive/MyDrive/Datasets/insulator_data/Defective_Insulators/images/000.jpg', cv2.IMREAD_COLOR)
img

array([[[217, 204, 206],
        [217, 204, 206],
        [217, 204, 206],
        ...,
        [127, 126, 135],
        [127, 127, 133],
        [129, 129, 135]],

       [[217, 204, 206],
        [217, 204, 206],
        [217, 204, 206],
        ...,
        [126, 125, 134],
        [126, 126, 132],
        [127, 127, 133]],

       [[217, 204, 206],
        [217, 204, 206],
        [217, 204, 206],
        ...,
        [126, 125, 134],
        [125, 125, 131],
        [126, 126, 132]],

       ...,

       [[107, 118,  98],
        [112, 123, 103],
        [115, 126, 106],
        ...,
        [145, 146, 130],
        [138, 141, 126],
        [136, 139, 124]],

       [[111, 122, 102],
        [113, 124, 104],
        [111, 122, 102],
        ...,
        [138, 139, 123],
        [135, 138, 123],
        [131, 134, 119]],

       [[113, 124, 104],
        [113, 124, 104],
        [111, 122, 102],
        ...,
        [131, 132, 116],
        [135, 138, 123],
        [135, 138, 123]]

In [ ]:
image = []                  # used to store image data
img_label = []              # used to store image label
img_indexes = []            # used to store image name
for i in range(248):
  img = cv2.imread('/content/gdrive/MyDrive/Datasets/insulator_data/Defective_Insulators/images/'+str(f'{i:03}')+'.jpg', cv2.IMREAD_COLOR)  
    # read image using opencv 
  img = cv2.resize(img, (224,224))                                                                   #resize image 
  img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
  image.append(img)
  img_label.append(0)                             #store label of each image in img_label
  img_indexes.append(f'{i:03}')

In [ ]:
image1 = []                  # used to store image data
img_label1 = []              # used to store image label
img_indexes1 = []            # used to store image name
for i in range(3617):
    if (os.path.exists("/content/gdrive/MyDrive/Datasets/insulator_data/Normal_Insulators/images/"+str(f'{i:04}')+'.jpg') == True):         # check if image exists or not
        img = cv2.imread('/content/gdrive/MyDrive/Datasets/insulator_data/Normal_Insulators/images/'+str(f'{i:04}')+'.jpg', cv2.IMREAD_COLOR)       # read image using opencv 
        img = cv2.resize(img, (224,224))                                                                   #resize image 
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        image1.append(img)
        img_label1.append(1)                             #store label of each image in img_label
        img_indexes1.append(f'{i:03}')
    else:
        pass

In [ ]:
final_img=np.array(image+image1)
final_label=np.array(img_label+img_label1)

In [ ]:
print(final_img.shape)
print(final_img.shape[1])
print(final_img.shape[2])
f = (final_img.shape[0], final_img.shape[2])
print(f)

(848, 224, 224, 3)
224
224
(848, 224)


In [ ]:
df = pd.DataFrame(list(zip(final_img, final_label)),columns =['Image','label'])
df.head()

,Image,label
0,"[[[206, 204, 217], [206, 204, 217], [206, 204,...",0
1,"[[[175, 184, 193], [176, 183, 193], [178, 185,...",0
2,"[[[102, 109, 107], [104, 108, 111], [108, 110,...",0
3,"[[[203, 207, 216], [203, 207, 216], [204, 208,...",0
4,"[[[157, 183, 154], [156, 183, 152], [159, 186,...",0


In [ ]:
a_f=[]
a_level=[]
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
])
for i in range(len(final_img)):
  augmented_image = data_augmentation(df["Image"][i])
  a_f.append(augmented_image)
  a_level.append(df["label"][i])
  

In [ ]:
len(a_f)

848

In [ ]:
a_level[0]

0

In [ ]:
def random_invert_img(x, p=0.5):
  if  tf.random.uniform([]) < p:
    x = (255-x)
  else:
    x
  return x

def random_invert(factor=0.5):
  return layers.Lambda(lambda x: random_invert_img(x, factor))

random_invert = random_invert()

In [ ]:
for i in range(len(final_img)):
  for i in range(9):
    augmented_image = random_invert(df["Image"][i])
    a_f.append(augmented_image)
    a_level.append(df["label"][i])

In [ ]:
len(a_f)

8480

In [ ]:
for i in range(len(final_img)):
  augmented_image = tf.image.flip_left_right(df["Image"][i])
  a_f.append(augmented_image)
  a_level.append(df["label"][i])


In [ ]:
len(a_f)

9328

In [ ]:
for i in range(len(final_img)):
  augmented_image = tf.image.rgb_to_grayscale(df["Image"][i])
  a_f.append(augmented_image)
  a_level.append(df["label"][i])

In [ ]:
len(a_f)

10176

In [ ]:
for i in range(len(final_img)):
  augmented_image =  tf.image.adjust_saturation(df["Image"][i], 3)
  a_f.append(augmented_image)
  a_level.append(df["label"][i])

In [ ]:
len(a_f)

11024

In [ ]:
for i in range(len(final_img)):
  for i in range(3):
    seed = (i,0)  # tuple of size (2,)
    augmented_image = tf.image.stateless_random_brightness(
    df["Image"][i], max_delta=0.8, seed=seed)
    a_f.append(augmented_image)
    a_level.append(df["label"][i])

In [ ]:
len(a_f)

13568